1. Simulate noisy (uniform, multiplicative) measurements from model with "true" parameters
2. Run Levenberg-Marquardt least squares optimisation to find "best fit" parameters (note the mismatch between noise and cost function types)
3. Run MBAM to find out if reduction is similar (both original and log-space assays)

Rinse and repeat

In [1]:
# Helper functions

# Get a timestamp in useful format
timestamp()=Dates.format(now(),"_yyyymmddTHHMMSSss")

timestamp (generic function with 1 method)

In [2]:
# Simulate data points with noise
include("../BK_functions/bk_setup_script.jl")
ydata_orig = BK_simulator(phi0, hcat(x_grid...), model_id=0) # Create dataset
ydata = ydata_orig.*(1+(2*rand(size(ydata_orig))-1)*0.1) # Add up to 10% noise (uniform, multiplicative)
;

In [3]:
# Optimise phi to fit the noisy data well
using LsqFit

# Store multiple fits (takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    phi_init = phi0.*(1+(2*rand(size(phi0))-1)*0.9) # Add up to 90% noise to initial params
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(p, x), hcat(x_grid...), ydata, phi_init);
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

phi1 = all_fits[findmin(all_costs)[2]].param

8-element Array{Float64,1}:
  3.34549e-6
  0.661848  
  0.134113  
  0.55403   
  4.96432e-5
  3.58497   
 19.6926    
  5.41314   

In [4]:
cur_time = timestamp();

a = "haho"*cur_time

"haho_20161108T15035400"

In [5]:
# Run MBAM from those parameters
cur_time = timestamp();

# First iteration:
outp = MBAM(BK_simulator, phi1, x_grid, model_iters=[10], 
move_dir=[-1], boundary_time=10, verbose=1, reduce_func=BK_reduce)

using JLD
save("saves/figure_A1"*cur_time*".jld", "outp", outp, "phi1", phi1, "ydata", ydata)

  0.006975 seconds (31.22 k allocations: 1.461 MB)
Model iteration 10, Single g_cost call takes: 
  0.036500 seconds (51.34 k allocations: 2.398 MB)
Model iteration 10, initial values:
      D = [  1.84e-05,   2.15e-04,   1.55e-03,   5.52e-02,   1.14e-01,   2.83e-01,   4.84e+00,   2.70e+01]
   cost = 4.8236526520437875e-30
phi_cur = [    -12.61,      -0.41,      -2.01,      -0.59,      -9.91,       1.28,       2.98,       1.69]
      v = [  9.62e-01,   1.90e-02,  -1.69e-02,   1.32e-02,   1.07e-02,   2.24e-02,  -2.70e-01,   8.83e-05]


Model iteration 10, final values:
      D = [  1.00e-08,   1.66e-04,   1.88e-03,   5.68e-02,   1.25e-01,   2.97e-01,   5.00e+00,   2.94e+01]
   cost = 0.00023628721026720863
phi_cur = [    -26.63,      -0.48,      -1.93,      -0.64,      -9.95,       1.20,       6.58,       1.68]
      v = [ -9.70e-01,  -3.93e-04,   5.98e-04,  -3.50e-04,  -3.15e-04,  -4.67e-04,   2.43e-01,  -1.79e-04]


